In [ ]:
import cv2
import time
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import warnings
import os
from tiki.mini import TikiMini
from ultralytics import YOLO

# 분리된 모듈 import
from utils.config import (
    BIRD_WIDTH, BIRD_HEIGHT, JSON_PATH
)
from utils.pid_controller import PIDController
from utils.server_communication import startSend, wait_for_json_file, send_dashboard_image
from utils.lane_detection import detect_lane_center, make_panel
from utils.pothole_detection import check_and_handle_pothole
from utils.image_utils import frame_to_bytes_drive, frame_to_bytes_yolo
from utils.aruco_detection import ArUcoDetector
from utils.yolo_detection_simpletrack import YOLODetector
from utils.camera_utils import init_camera, init_perspective_transform
from utils.driving_control import control_motors
from utils.qr_detection import QRDetector
from utils.led_control import LEDController

warnings.filterwarnings('ignore', category=UserWarning)
os.environ['GST_DEBUG'] = '0'

##############################################
# 1. YOLOv8 학습 모델 로드 (가장 먼저 실행)
##############################################
model = YOLO("./best4.pt")  # 🔥 학습한 best.pt 경로 확인
print("[YOLO] 모델 로드 완료")


##############################################

##############################################
# 2. ArUco 설정
##############################################
aruco_detector = ArUcoDetector()

##############################################
# 3. 로봇 초기화
##############################################
tiki = TikiMini()
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)
# 안전: 초기화 직후 모터 정지
try:
    tiki.stop()
except Exception:
    pass
print("로봇 초기화 완료 (모터 정지)")

# PID 제어
pid = PIDController()

##############################################
# 4. 단일 카메라 파이프라인 (주행 + YOLO 공용)
##############################################
cap = init_camera()

##############################################
# 5. Jupyter용 두 화면 위젯 (주행 / 객체인식)
##############################################
drive_widget = widgets.Image(
    format='jpeg',
    layout=widgets.Layout(width='640px', height='480px')
)
yolo_widget = widgets.Image(
    format='jpeg',
    layout=widgets.Layout(width='640px', height='480px')
)

display(widgets.HBox([drive_widget, yolo_widget]))

##############################################
# 6. Bird's-Eye View 설정 (주행용)
##############################################
M = init_perspective_transform()

##############################################
# 6-1. YOLO 감지기 초기화
##############################################
yolo_detector = YOLODetector(model, aruco_detector)

##############################################
# 6-2. QR 코드 감지기 및 LED 제어기 초기화
##############################################
qr_detector = QRDetector()
led_controller = LEDController(tiki)
print("[QR/LED] QR 감지기 및 LED 제어기 초기화 완료")

# 1-2. 시작 통신 전송 (startdashboard 로직)
startSend()
# ##############################################

##############################################

##############################################


##############################################
# 7. 메인 루프 (단일 카메라 → 주행 + YOLO 두 화면 출력)
##############################################
print("[START] 주행 + YOLO + ArUco 시스템 실행")

frame_idx = 0
annotated = None  # 마지막 YOLO 결과 프레임 보관
qr_detected = False  # QR 코드 인식 플래그 (한 번만 실행)
timer = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            time.sleep(8)
            continue

        frame_idx += 1

        # 공통 원본 프레임 (카메라 기준 회전 보정)
        frame = cv2.flip(frame, -1)

        # 주행용 처리를 위해 320x240으로 다운스케일한 프레임 사용
        frame_small = cv2.resize(frame, (BIRD_WIDTH, BIRD_HEIGHT))


        # ----- (1) 주행용 버드뷰 / 차선 인식 -----
        bird = cv2.warpPerspective(frame_small, M, (BIRD_WIDTH, BIRD_HEIGHT))
        center_x, is_horizontal_bar, dbg = detect_lane_center(bird)
        view = bird.copy()

        # ----- (2) 객체 인식 (YOLO + ArUco) : N프레임마다만 수행 -----
        frame_for_aruco = frame.copy()

        # ---------------------------
        # (A) ArUco 감지
        # ---------------------------
        corners, ids, rejected = aruco_detector.detect(frame)
        point_detected, sector_detected, sector_name = aruco_detector.process_markers(frame, ids, corners)

        # ---------------------------
        # (B) sector 발견 → 회전 + 사진 촬영 + 방향 복구 + 서버 전송
        # ---------------------------
        if sector_detected:
            print(f"[SECTOR] {sector_name} 감지됨 → 회전 후 촬영")

            # 숫자 추출
            import re
            num = int(re.search(r"\d+", sector_name).group())

            # 회전 시간
            ROTATE_TIME = 0.5
            SPEED = 40

            # 1) 회전 방향 결정
            if num in [1, 2, 3, 4]:
                print("[ACTION] RIGHT 회전")
                tiki.clockwise(SPEED)
                time.sleep(ROTATE_TIME)
                rotate_dir = "right"
            else:
                print("[ACTION] LEFT 회전")
                tiki.counterclockwise(SPEED)
                time.sleep(ROTATE_TIME)
                rotate_dir = "left"

            # 2) 회전 안정화 대기
            time.sleep(ROTATE_TIME)
            tiki.stop()

            # 2) 촬영
            ret2, frame2 = cap.read()
            if ret2:
                filename = f"{mission_code}_sector{num}.jpg"
                image_path = f"/tmp/{filename}"

                cv2.imwrite(image_path, frame2)
                print(f"[CAPTURE] 촬영 완료 → {image_path}")

                send_dashboard_image(image_path)
                print(f"[UPLOAD] 서버 전송 완료")

            # ----------------------------------------
            # 4) 원래 방향으로 복귀 (역회전)
            # ----------------------------------------
            print("[ACTION] 원래 방향으로 복귀 중")

            if rotate_dir == "right":
                tiki.counterclockwise(SPEED)
                time.sleep(ROTATE_TIME)
            else:
                tiki.clockwise(SPEED)
                time.sleep(ROTATE_TIME)

            time.sleep(ROTATE_TIME)
            tiki.stop()

            print("[ACTION] 복귀 완료 → 주행 재개")


        # (B) YOLO 객체 감지
        annotated, detected_counts = yolo_detector.detect_objects(frame, frame_idx)

        # ----- (3) 두 화면을 Jupyter 위젯으로 출력 -----
        panel = make_panel(view, dbg)
        drive_widget.value = frame_to_bytes_drive(panel)
        if annotated is not None:
            yolo_widget.value = frame_to_bytes_yolo(annotated)

        # ----- (4) 주행 시작 직후 QR 코드 인식 및 LED 제어 (한 번만 실행) -----
        
        
        # 1-1. A3R8.json 파일 대기 (파일이 생성될 때까지 무한 대기)
        ##############################################
        test_data = wait_for_json_file()
        
        # QR코드 인식. 처음 루프에서 json감지 된 후 2초 지났을때 멈춰서 QR인식
        if not qr_detected and timer == 0:
            import time
            timer = time.time()
        if not qr_detected and timer != 0 and (time.time() - timer > 2): #2초지났을때
            tiki.stop()
            time.sleep(2)
            qr_data = qr_detector.detect_from_frame(frame, led_controller)
            qr_detected = True #첫 루프에서만 실행되도록 플래그 설정
        # 주행 제어
        control_motors(tiki, center_x, is_horizontal_bar, pid)
                    
        # ----- 포트홀 감지 (적응형) -----
        check_and_handle_pothole(bird, tiki)


        time.sleep(0.02)

except KeyboardInterrupt:
    tiki.stop()
    cap.release()
    cv2.destroyAllWindows()
    print("[END] 프로그램 종료")
